In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from scipy.spatial.transform import Rotation


### 读入图像中检测的点坐标

In [2]:
all_pts_2d = np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/detected_corners.txt")

In [5]:
row, column = all_pts_2d.shape
print(row)
print(column)

210
24


In [6]:
all_pts_2d

array([[186.5, 187.5, 264.5, ..., 280.5, 496.5, 300.5],
       [186.5, 186.5, 264.5, ..., 279.5, 496.5, 300.5],
       [187.5, 186.5, 265.5, ..., 279.5, 496.5, 300.5],
       ...,
       [220.5, 216.5, 293.5, ..., 306.5, 547.5, 319.5],
       [214.5, 226.5, 289.5, ..., 313.5, 542.5, 327.5],
       [208.5, 236.5, 283.5, ..., 322.5, 537.5, 336.5]])

### 对图像点坐标进行reshape，获取一组图像的中检测到的点

In [10]:
image_idx = 1
pts_2d = np.reshape(
    np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/detected_corners.txt")[image_idx-1,:],
    (-1, 2) )

In [11]:
pts_2d.shape

(12, 2)

In [12]:
pts_2d

array([[186.5, 187.5],
       [264.5, 166.5],
       [218.5, 304.5],
       [285.5, 270.5],
       [292.5, 369.5],
       [356.5, 322.5],
       [406.5, 418.5],
       [461.5, 353.5],
       [388.5, 204.5],
       [495.5, 211.5],
       [396.5, 280.5],
       [496.5, 300.5]])

In [7]:
p1 = all_pts_2d[1,:]

In [13]:
pts_2d = np.reshape(all_pts_2d[0,:],(-1,2))

In [15]:
pts_2d

array([[186.5, 187.5],
       [264.5, 166.5],
       [218.5, 304.5],
       [285.5, 270.5],
       [292.5, 369.5],
       [356.5, 322.5],
       [406.5, 418.5],
       [461.5, 353.5],
       [388.5, 204.5],
       [495.5, 211.5],
       [396.5, 280.5],
       [496.5, 300.5]])

In [17]:
# 内参矩阵
K = np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/K.txt")

In [18]:
K

array([[420.506712,   0.      , 355.208298],
       [  0.      , 420.61094 , 250.336787],
       [  0.      ,   0.      ,   1.      ]])

In [19]:
# 点的世界坐标系
p_W_corners = 0.01 * np.loadtxt("/home/mullin/WorkSpace/CourseProject/3 VAMR/Exercise 2 - PnP/data/p_W_corners.txt", delimiter = ",")

In [20]:
p_W_corners

array([[ 0.    ,  0.08  ,  0.    ],
       [ 0.104 ,  0.08  ,  0.    ],
       [ 0.    ,  0.215 ,  0.    ],
       [ 0.104 ,  0.215 ,  0.    ],
       [ 0.    ,  0.266 , -0.1095],
       [ 0.104 ,  0.266 , -0.1095],
       [ 0.    ,  0.266 , -0.2445],
       [ 0.104 ,  0.266 , -0.2445],
       [ 0.194 ,  0.146 , -0.1095],
       [ 0.194 ,  0.146 , -0.2445],
       [ 0.194 ,  0.25  , -0.1095],
       [ 0.194 ,  0.25  , -0.2445]])

In [60]:
# 世界坐标与图像坐标下的点
p = pts_2d
P = p_W_corners

### 计算在相机坐标系下的坐标

In [64]:
p_norm = (np.linalg.inv(K) @ np.c_[p, np.ones((p.shape[0], 1))].T).T

In [23]:
p_norm

array([[-0.40120239, -0.14939409,  1.        ],
       [-0.21571189, -0.19932146,  1.        ],
       [-0.32510372,  0.12877272,  1.        ],
       [-0.16577214,  0.04793792,  1.        ],
       [-0.14912556,  0.28330983,  1.        ],
       [ 0.00307177,  0.17156761,  1.        ],
       [ 0.12197594,  0.39980704,  1.        ],
       [ 0.25277052,  0.24526992,  1.        ],
       [ 0.07917044, -0.10897669,  1.        ],
       [ 0.33362536, -0.09233423,  1.        ],
       [ 0.09819511,  0.07171286,  1.        ],
       [ 0.33600344,  0.11926274,  1.        ]])

In [49]:
np.c_[p, np.ones((p.shape[0], 1))].T

array([[186.5, 264.5, 218.5, 285.5, 292.5, 356.5, 406.5, 461.5, 388.5,
        495.5, 396.5, 496.5],
       [187.5, 166.5, 304.5, 270.5, 369.5, 322.5, 418.5, 353.5, 204.5,
        211.5, 280.5, 300.5],
       [  1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ]])

In [52]:
pts_2d_aug = np.c_[p, np.ones((p.shape[0], 1))]

In [53]:
pts_2d_aug

array([[186.5, 187.5,   1. ],
       [264.5, 166.5,   1. ],
       [218.5, 304.5,   1. ],
       [285.5, 270.5,   1. ],
       [292.5, 369.5,   1. ],
       [356.5, 322.5,   1. ],
       [406.5, 418.5,   1. ],
       [461.5, 353.5,   1. ],
       [388.5, 204.5,   1. ],
       [495.5, 211.5,   1. ],
       [396.5, 280.5,   1. ],
       [496.5, 300.5,   1. ]])

In [72]:
p_cam = (np.mat(K).I * pts_2d_aug.T).T [:,0:2]

In [73]:
p_cam

matrix([[-0.40120239, -0.14939409],
        [-0.21571189, -0.19932146],
        [-0.32510372,  0.12877272],
        [-0.16577214,  0.04793792],
        [-0.14912556,  0.28330983],
        [ 0.00307177,  0.17156761],
        [ 0.12197594,  0.39980704],
        [ 0.25277052,  0.24526992],
        [ 0.07917044, -0.10897669],
        [ 0.33362536, -0.09233423],
        [ 0.09819511,  0.07171286],
        [ 0.33600344,  0.11926274]])

In [78]:
p_cam [0,0]

-0.40120239031999094

### 获取Q矩阵

In [75]:
p_W_mat = np.mat(p_W_corners)
p_W_mat_h = np.hstack((p_W_mat,np.mat(np.ones((12,1)))))

In [76]:
p_W_mat_h

matrix([[ 0.    ,  0.08  ,  0.    ,  1.    ],
        [ 0.104 ,  0.08  ,  0.    ,  1.    ],
        [ 0.    ,  0.215 ,  0.    ,  1.    ],
        [ 0.104 ,  0.215 ,  0.    ,  1.    ],
        [ 0.    ,  0.266 , -0.1095,  1.    ],
        [ 0.104 ,  0.266 , -0.1095,  1.    ],
        [ 0.    ,  0.266 , -0.2445,  1.    ],
        [ 0.104 ,  0.266 , -0.2445,  1.    ],
        [ 0.194 ,  0.146 , -0.1095,  1.    ],
        [ 0.194 ,  0.146 , -0.2445,  1.    ],
        [ 0.194 ,  0.25  , -0.1095,  1.    ],
        [ 0.194 ,  0.25  , -0.2445,  1.    ]])

In [79]:
num_corners = p.shape[0]

In [80]:
num_corners

12

In [81]:
column

24

In [77]:
Q = np.mat(np.zeros((num_corners*2,12)))

In [83]:
Q = np.mat(np.zeros((num_corners*2,12)))
for i in range (num_corners):
    Q[2*i, 0:4] = p_W_mat_h[i]
    Q[2*i, 8:12] = p_W_mat_h[i] * (-1) * p_cam[i,0]
    Q[2*i+1, 4:8] = p_W_mat_h[i]
    Q[2*i+1, 8:12] = p_W_mat_h[i] * (-1) * p_cam[i,1]

In [87]:
Q[0,:]

matrix([[0.        , 0.08      , 0.        , 1.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.03209619,
         0.        , 0.40120239]])

In [85]:
    Q_ = np.zeros((2*num_corners, 12))
    for i in range(num_corners):
        u = p_norm[i, 0]
        v = p_norm[i, 1]

        Q_[2*i, 0:3] = P[i,:]
        Q_[2*i, 3] = 1
        Q_[2*i, 8:11] = -u * P[i,:]
        Q_[2*i, 11] = -u
        
        Q_[2*i+1, 4:7] = P[i,:]
        Q_[2*i+1, 7] = 1
        Q_[2*i+1, 8:11] = -v * P[i,:]
        Q_[2*i+1, 11] = -v

In [88]:
Q_[0,:]

array([0.        , 0.08      , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03209619,
       0.        , 0.40120239])

### SVD

In [132]:
U,sigma, VT = np.linalg.svd(Q,full_matrices=True)

In [133]:
VT

matrix([[ 1.22043317e-02,  2.32921933e-02, -1.51884999e-02,
          1.10814842e-01,  9.54536943e-02,  1.97370856e-01,
         -1.15682008e-01,  9.57663812e-01, -4.26150745e-03,
         -2.83140007e-02,  2.05493970e-02, -8.35872029e-02],
        [ 9.63009996e-02,  1.97190986e-01, -1.15361032e-01,
          9.61592675e-01, -1.11730362e-02, -2.29282394e-02,
          1.34123508e-02, -1.11641480e-01, -1.22378377e-02,
         -3.70251255e-03,  1.91988476e-02,  6.84630543e-03],
        [-1.29601367e-01, -6.03102259e-02,  2.27919388e-01,
          6.02570523e-02,  6.55953355e-02, -9.04714385e-02,
          7.70667577e-02,  1.09334628e-01,  7.05667483e-02,
          1.72028808e-01, -1.16653777e-01,  9.19470036e-01],
        [ 1.17319426e-01,  1.03178884e-01, -2.77777264e-01,
         -6.53674243e-02, -3.33794326e-01, -2.93706868e-01,
          8.10967723e-01,  1.91575612e-01, -1.00579593e-02,
          1.01203136e-02, -4.31900527e-02, -5.99646731e-03],
        [-4.42204880e-02, -3.1914118

In [136]:
M = VT.T[:,-1].reshape(3,4)
if (np.linalg.det(M[:, :3]) < 0):
    M *= -1

In [137]:
M

matrix([[ 0.34576372,  0.0720053 , -0.435178  , -0.10223635],
        [-0.19232431,  0.52029977, -0.06663463, -0.07753862],
        [ 0.38837535,  0.19694482,  0.34239943,  0.22453859]])

In [138]:
R = M[0:3,0:3]

In [139]:
R

matrix([[ 0.34576372,  0.0720053 , -0.435178  ],
        [-0.19232431,  0.52029977, -0.06663463],
        [ 0.38837535,  0.19694482,  0.34239943]])

In [140]:
U_R, Sigma_R, VT_R = np.linalg.svd(R)

In [141]:
R_ = U_R*VT_R

In [142]:
R_

matrix([[ 0.61751389,  0.12882547, -0.77593852],
        [-0.34983735,  0.92853417, -0.12425027],
        [ 0.70447883,  0.34817854,  0.61845071]])

In [146]:
alpha = np.linalg.norm(R_) / np.linalg.norm(R)

In [147]:
alpha

1.7930498521818585

In [158]:
M_ = np.c_[R_, alpha * M[:,3]];

In [159]:
M_

matrix([[ 0.61751389,  0.12882547, -0.77593852, -0.18331488],
        [-0.34983735,  0.92853417, -0.12425027, -0.13903062],
        [ 0.70447883,  0.34817854,  0.61845071,  0.40260889]])

### answer

In [150]:
    u, s, v = np.linalg.svd(Q, full_matrices=True)
    M_tilde = np.reshape(v.T[:,-1], (3,4));
    
    # Extract [R | t] with the correct scale
    if (np.linalg.det(M_tilde[:, :3]) < 0):
        M_tilde *= -1


In [151]:
M_tilde

matrix([[ 0.34576372,  0.0720053 , -0.435178  , -0.10223635],
        [-0.19232431,  0.52029977, -0.06663463, -0.07753862],
        [ 0.38837535,  0.19694482,  0.34239943,  0.22453859]])

In [152]:
    R = M_tilde[:, :3]

In [153]:
R

matrix([[ 0.34576372,  0.0720053 , -0.435178  ],
        [-0.19232431,  0.52029977, -0.06663463],
        [ 0.38837535,  0.19694482,  0.34239943]])

In [154]:
    # Find the closest orthogonal matrix to R
    # https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
    u, s, v = np.linalg.svd(R);
    R_tilde = u @ v;

    # Normalization scheme using the Frobenius norm:
    # recover the unknown scale using the fact that R_tilde is a true rotation matrix
    alpha = np.linalg.norm(R_tilde, 'fro')/np.linalg.norm(R, 'fro');

    # Build M_tilde with the corrected rotation and scale
    M_tilde = np.c_[R_tilde, alpha * M_tilde[:,3]];

In [155]:
R_tilde

matrix([[ 0.61751389,  0.12882547, -0.77593852],
        [-0.34983735,  0.92853417, -0.12425027],
        [ 0.70447883,  0.34817854,  0.61845071]])

In [156]:
alpha

1.7930498521818585

In [157]:
M_tilde

matrix([[ 0.61751389,  0.12882547, -0.77593852, -0.18331488],
        [-0.34983735,  0.92853417, -0.12425027, -0.13903062],
        [ 0.70447883,  0.34817854,  0.61845071,  0.40260889]])

### Function

In [166]:
def estimatePoseDLT(p, P, K):
    
    num_corners = p.shape[0]
    
    pts_2d_aug = np.c_[p, np.ones((p.shape[0], 1))]
    p_cam = (np.mat(K).I * pts_2d_aug.T).T [:,0:2]
    p_W_mat_h = np.hstack((P,np.mat(np.ones((num_corners,1)))))
    

    Q = np.mat(np.zeros((num_corners*2,12)))
    for i in range (num_corners):
        Q[2*i, 0:4] = p_W_mat_h[i]
        Q[2*i, 8:12] = p_W_mat_h[i] * (-1) * p_cam[i,0]
        Q[2*i+1, 4:8] = p_W_mat_h[i]
        Q[2*i+1, 8:12] = p_W_mat_h[i] * (-1) * p_cam[i,1]
        
    U,sigma, VT = np.linalg.svd(Q,full_matrices=True)
    M = VT.T[:,-1].reshape(3,4)
    if (np.linalg.det(M[:, :3]) < 0):
        M *= -1    
    R = M[0:3,0:3]
    U_R, Sigma_R, VT_R = np.linalg.svd(R)
    R_ = U_R*VT_R
    alpha = np.linalg.norm(R_) / np.linalg.norm(R)
    M_ = np.c_[R_, alpha * M[:,3]]
    
    return M_

In [167]:
M = estimatePoseDLT(p, P, K)

In [168]:
print

<function print>

In [169]:
M

matrix([[ 0.61751389,  0.12882547, -0.77593852, -0.18331488],
        [-0.34983735,  0.92853417, -0.12425027, -0.13903062],
        [ 0.70447883,  0.34817854,  0.61845071,  0.40260889]])